<h3>Can Skip</h3>

In [1]:
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI

from typing import List
from langchain.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from pydantic import BaseModel, Field
import json
from langchain_core.agents import AgentActionMessageLog,AgentFinish
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
llmGemini=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llmOpenAI=ChatOpenAI(model="gpt-4o-mini")

In [4]:
# Load Documents

loader=TextLoader(file_path="data/state_of_the_union.txt",encoding="utf-8")
documents=loader.load()

In [5]:
# Split Documents into Chunks
textSplitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts=textSplitter.split_documents(documents=documents)

In [6]:
# Illustration
texts[4]

Document(metadata={'source': 'data/state_of_the_union.txt'}, page_content='And tonight I am announcing that we will join our allies in closing off American air space to all Russian flights – further isolating Russia – and adding an additional squeeze –on their economy. The Ruble has lost 30% of its value. \n\nThe Russian stock market has lost 40% of its value and trading remains suspended. Russia’s economy is reeling and Putin alone is to blame. \n\nTogether with our allies we are providing support to the Ukrainians in their fight for freedom. Military assistance. Economic assistance. Humanitarian assistance. \n\nWe are giving more than $1 Billion in direct assistance to Ukraine. \n\nAnd we will continue to aid the Ukrainian people as they defend their country and to help ease their suffering.  \n\nLet me be clear, our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  \n\nOur forces are not going to Europe to fight in Ukraine, but to defend our NAT

In [8]:
# Illustration
texts[4].metadata["page_chunk"]=4

In [8]:
# Illustration
texts[4]

Document(metadata={'source': 'data/state_of_the_union.txt', 'page_chunk': 4}, page_content='And tonight I am announcing that we will join our allies in closing off American air space to all Russian flights – further isolating Russia – and adding an additional squeeze –on their economy. The Ruble has lost 30% of its value. \n\nThe Russian stock market has lost 40% of its value and trading remains suspended. Russia’s economy is reeling and Putin alone is to blame. \n\nTogether with our allies we are providing support to the Ukrainians in their fight for freedom. Military assistance. Economic assistance. Humanitarian assistance. \n\nWe are giving more than $1 Billion in direct assistance to Ukraine. \n\nAnd we will continue to aid the Ukrainian people as they defend their country and to help ease their suffering.  \n\nLet me be clear, our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  \n\nOur forces are not going to Europe to fight in Ukraine, but 

In [9]:
# Add in the fake source information (how is covered in the illustrations above)
for i, doc in enumerate(texts):
    doc.metadata["page_chunk"]=i

In [10]:
# Create Vectorstore and retriever
embeddings=OpenAIEmbeddings()
vectorstore=FAISS.from_documents(documents=texts,embedding=embeddings)
retriever=vectorstore.as_retriever()

In [11]:
# Create Retriever Tool

retrieverTool=create_retriever_tool(
    retriever=retriever,
    name="state-of-union-retriever",
    description="Query a retriever to get information about state of the union address"
)

In [12]:
class Response(BaseModel):
    """
        Final response to the question being asked
    """
    answer: str=Field(
        description="The final answer to respond to the user"
    )

    sources: List[int]=Field(
        description="""
            List of page chunks that contain answer to the question.
            Only include a page chunk if it contains a relevant information
        """
    )

In [13]:
llmWithTools=llmOpenAI.bind_tools(tools=[retrieverTool,Response])

In [14]:
prompt=ChatPromptTemplate.from_messages(
    messages=[
        ("system","You are a helpful assistant"),
        ("user","{input}"),
        ("placeholder","{agent_scratchpad}")
    ]
)

In [ ]:
def parse(output):
    # If no function was involved, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(
            return_values={"output":output.content},
            log=output.content
        )
    # Else Parse out the content
    else:
        function_call=output.additional_kwargs['function_call']
        name=function_call['name']
        inputs=json.loads(function_call['arguments'])

        # If the Response Function was invoked, return to the user with the function inputs
        if name=="Response":
            return AgentFinish(return_values=inputs,log=str(function_call))
        # if any other function is called, treat it as a tool invocation and return an agent action
        else:
            return AgentActionMessageLog(tool=name, tool_input=inputs, log="",message_log=[output])
            

In [22]:
# Creating the Custom Agent
agent={
    "input":lambda x:x['input'],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(x['intermediate_steps'])
    } | prompt | llmWithTools | parse

In [23]:
agentExecutor=AgentExecutor(agent=agent, tools=[retrieverTool],verbose=True)

In [24]:
agentExecutor.invoke(
    input={"input":"What did the President say about Putin?"},
)



> Entering new AgentExecutor chain...


> Finished chain.


{'input': 'What did the President say about Putin?', 'output': ''}

In [25]:
agentExecutor.invoke(
    input={"input":"What did the President say about Ketanji brown Jackson?"},
)



> Entering new AgentExecutor chain...


> Finished chain.


{'input': 'What did the President say about Ketanji brown Jackson?',
 'output': ''}

In [26]:
for content in agentExecutor.iter(
    inputs={"input":"What did the President say about Putin?"},
    
):
    print(content,"-\n------")



> Entering new None chain...


> Finished chain.
{'output': '', 'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={})]} -
------
